# Acoustic Experiments

## Imports

In [1]:
%matplotlib inline

import math
import numpy as np
import scipy as sp
import scipy.constants as const
import pandas as pd

from scipy.io import wavfile
from scipy import signal
from scipy.signal import butter, firwin, lfilter, freqz, argrelmax

## Constants & Global Variables

In [2]:
input_data_directory = "data"
output_data_directory = "out"

sample_rate = 44100
tone_duration = 0.05
tone_frequency0 = 2000
tone_frequency1 = 6000

def normalize_data(data):
    return data/0x7FFF

def filter_data(data,
                low_cut_frequency,
                high_cut_frequency,
                filter_order,
                filter_type=None,
                filter_parameters=None,
                sample_rate=44100):
    nyquist_frequency = sample_rate/2
    if filter_type is None or filter_type == "none":
        return data
    elif filter_type == "firwin":
        b = firwin(filter_order+1,
                   [low_cut_frequency, high_cut_frequency],
                   nyq=nyquist_frequency,
                   pass_zero=False)
        a = [1.0]
    elif filter_type == "butterworth":
        b, a = butter(filter_order,
                      [low_cut_frequency/nyquist_frequency, high_cut_frequency/nyquist_frequency],
                      btype="bandpass")
    else:
        raise NameError("Unsupported Filter Type")
    return lfilter(b, a, data)

## Tone Generation

In [3]:
def sinwave_tone(frequency, duration, sample_rate):
    return np.sin(2 * math.pi * np.arange(math.ceil(sample_rate * duration)) * frequency / sample_rate) * 0x7FFF

def chirp_tone(frequency0, frequency1, duration, phase0=0, sample_rate=44100):
    num_samples = math.ceil(sample_rate * duration);
    k = (frequency1 - frequency0)/num_samples
    return np.sin(phase0+2*math.pi*(np.arange(num_samples)*frequency0/sample_rate+k/2*np.arange(num_samples)**2/sample_rate)) * 0x7FFF

## Helper Functions

In [4]:
def generate_tone(tone_frequency0, 
                  tone_frequency1,
                  tone_duration,
                  sample_rate,
                  filter_order,
                  filter_type,
                  save=True,
                  path=input_data_directory+"/audio/tone.wav"):
    tone_data = chirp_tone(tone_frequency0, tone_frequency1, tone_duration, sample_rate)
    if save:
        wavfile.write(path, sample_rate, tone_data.astype(np.int16))
    tone_data = filter_data(normalize_data(tone_data), tone_frequency0, tone_frequency1, filter_order, filter_type)
    return tone_data

def load_sample(path, tone_frequency0, tone_frequency1, filter_order, filter_type):
    sample_rate, sample_data = wavfile.read(path)
    return sample_rate, filter_data(normalize_data(sample_data), tone_frequency0, tone_frequency1, filter_order, filter_type)
       
def tone_correlation(sample_data, tone_frequency0, tone_frequency1, tone_data):
    correlation = sp.correlate(sample_data, tone_data, "valid")
    return correlation

def tone_recognition(sample_data, sample_rate, duration):
    last_index = 0
    tones = []
    tone_candidates = argrelmax(sample_data, order=math.ceil(duration*sample_rate/2))[0]
    for candidate in tone_candidates:
        tones.append((candidate, sample_data[candidate]))
    tones.sort(key=lambda tup: tup[1])
    tones.reverse()
    
    return tones

def compute_etoa(tones, sample_rate):
    if len(tones) < 2:
        raise NameError("Insufficient Tones Detected")
    else:
        return abs(tones[0][0]-tones[1][0])/sample_rate
    
def compute_distance(etoa_a, etoa_b, d_aa, d_bb, c=340.29):
    return (c/2) * (etoa_a - etoa_b) + (d_aa + d_bb)

def device_distance(self_distance_a,
                    self_distance_b,
                    device_a_path,
                    device_b_path,
                    tone_frequency0,
                    tone_frequency1,
                    tone_duration,
                    sample_rate,
                    filter_order,
                    filter_type,
                    verbose=False):

    tone_data = generate_tone(tone_frequency0,
                              tone_frequency1,
                              tone_duration,
                              sample_rate,
                              filter_order,
                              filter_type)
    if verbose:
        plt.figure()
        plt.plot(tone_data)
    
    
    if verbose:
        print("Device A Sample:", device_a_path)
        print("Device B Sample:", device_b_path)
    
    
    device_a_sample_rate, device_a_sample_data = load_sample(device_a_path,
                                                             tone_frequency0,
                                                             tone_frequency1,
                                                             filter_order,
                                                             filter_type)
    if verbose:
        plt.figure()
        plt.suptitle("Device A Waveform", fontsize=16)
        plt.plot(device_a_sample_data)
    
    device_b_sample_rate, device_b_sample_data = load_sample(device_b_path,
                                                             tone_frequency0,
                                                             tone_frequency1,
                                                             filter_order,
                                                             filter_type)
    if verbose:
        plt.figure()
        plt.suptitle("Device B Waveform", fontsize=16)
        plt.plot(device_b_sample_data)
    
    
    device_a_correlation = tone_correlation(device_a_sample_data,
                                            tone_frequency0,
                                            tone_frequency1,
                                            tone_data)
    if verbose:
        plt.figure()
        plt.suptitle("Device A Correlation", fontsize=16)
        plt.plot(device_a_correlation)
    
    device_b_correlation = tone_correlation(device_b_sample_data,
                                            tone_frequency0,
                                            tone_frequency1,
                                            tone_data)
    if verbose:
        plt.figure()
        plt.suptitle("Device B Correlation", fontsize=16)
        plt.plot(device_b_correlation)
    
    
    device_a_tones = tone_recognition(device_a_correlation, sample_rate, tone_duration)
    if verbose:
        print("Device A")
        print("A total of",len(device_a_tones),"tones were detected.")
        print("The best candidates are (",
              device_a_tones[0][0],",",device_a_tones[0][1],") and (",
              device_a_tones[1][0],",",device_a_tones[1][1],")")  
        print("A total of",len(device_a_tones),"tones were detected.")    
    
    device_b_tones = tone_recognition(device_b_correlation, sample_rate, tone_duration)
    if verbose: 
        print("Device B")
        print("The best candidates are (",
              device_b_tones[0][0],",",device_b_tones[0][1],") and (",
              device_b_tones[1][0],",",device_b_tones[1][1],")")  
        print("A total of",len(device_b_tones),"tones were detected.")
    
    
    etoa_a = compute_etoa(device_a_tones, sample_rate)
    if verbose: 
        print("Device A ETOA:",device_a_etoa)
        
    etoa_b = compute_etoa(device_b_tones, sample_rate)
    if verbose:
        print("Device B ETOA:",device_b_etoa)

    
    distance = compute_distance(etoa_a, etoa_b, self_distance_a, self_distance_b)
    if verbose:
        print("Distance between devices in meters:", distance)
    return distance

## Trials

In [6]:
self_distance_a = 0.04
self_distance_b = 0.04

trials = [
            {"actual_distance": 0.5,
             "self_distance_a": self_distance_a,
             "self_distance_b": self_distance_b,
             "device_a_sample_path": input_data_directory+"/audio/trial-0-chirp-0.5m-da.wav",
             "device_b_sample_path": input_data_directory+"/audio/trial-0-chirp-0.5m-db.wav"},
    
            {"actual_distance": 0.5,
             "self_distance_a": self_distance_a,
             "self_distance_b": self_distance_b,
             "device_a_sample_path": input_data_directory+"/audio/trial-1-chirp-0.5m-da.wav",
             "device_b_sample_path": input_data_directory+"/audio/trial-1-chirp-0.5m-db.wav"},
    
            {"actual_distance": 0.5,
             "self_distance_a": self_distance_a,
             "self_distance_b": self_distance_b,
             "device_a_sample_path": input_data_directory+"/audio/trial-2-chirp-0.5m-da.wav",
             "device_b_sample_path": input_data_directory+"/audio/trial-2-chirp-0.5m-db.wav"},
    
            {"actual_distance": 1,
             "self_distance_a": self_distance_a,
             "self_distance_b": self_distance_b,
             "device_a_sample_path": input_data_directory+"/audio/trial-0-chirp-1m-da.wav",
             "device_b_sample_path": input_data_directory+"/audio/trial-0-chirp-1m-db.wav"},
    
            {"actual_distance": 1,
             "self_distance_a": self_distance_a,
             "self_distance_b": self_distance_b,
             "device_a_sample_path": input_data_directory+"/audio/trial-1-chirp-1m-da.wav",
             "device_b_sample_path": input_data_directory+"/audio/trial-1-chirp-1m-db.wav"},
    
            {"actual_distance": 1,
             "self_distance_a": self_distance_a,
             "self_distance_b": self_distance_b,
             "device_a_sample_path": input_data_directory+"/audio/trial-2-chirp-1m-da.wav",
             "device_b_sample_path": input_data_directory+"/audio/trial-2-chirp-1m-db.wav"},
]

filter_orders = [3,6,9,12]
filter_types = ["firwin","butterworth","none"]
results = []

for trial in trials:
    for filter_order in filter_orders:
        for filter_type in filter_types:
            print("Trial Files:",trial["device_a_sample_path"],"and",trial["device_b_sample_path"])
            print("Filter Order:",filter_order)
            print("Filter Type:",filter_type)
            print("Actual distance between devices in meters:",trial["actual_distance"])
            estimated_distance = device_distance(trial["self_distance_a"],
                                                 trial["self_distance_b"],
                                                 trial["device_a_sample_path"],
                                                 trial["device_b_sample_path"],
                                                 tone_frequency0,
                                                 tone_frequency1,
                                                 tone_duration,
                                                 sample_rate,
                                                 filter_order,
                                                 filter_type,
                                                 verbose=False)
            
            print("Estimated distance between devices in meters:", estimated_distance)
            absolute_error = abs(estimated_distance-trial["actual_distance"])
            print("Absolute error:",absolute_error)
            results.append({"actual_distance": trial["actual_distance"],
                            "self_distance_a": trial["self_distance_a"],
                            "self_distance_b": trial["self_distance_b"],
                            "device_a_sample_path": trial["device_a_sample_path"],
                            "device_b_sample_path": trial["device_b_sample_path"],
                            "tone_frequency0": tone_frequency0,
                            "tone_frequency1": tone_frequency1,
                            "tone_duration": tone_duration,
                            "sample_rate": sample_rate,
                            "filter_order": filter_order,
                            "filter_type": filter_type})            
            print("Relative error:", absolute_error/trial["actual_distance"])
            print("--------------------")

Trial Files: data/audio/trial-0-chirp-0.5m-da.wav and data/audio/trial-0-chirp-0.5m-db.wav
Filter Order: 3
Filter Type: firwin
Actual distance between devices in meters: 0.5
Estimated distance between devices in meters: 0.52368877551
Absolute error: 0.0236887755102
Relative error: 0.0473775510204
--------------------
Trial Files: data/audio/trial-0-chirp-0.5m-da.wav and data/audio/trial-0-chirp-0.5m-db.wav
Filter Order: 3
Filter Type: butterworth
Actual distance between devices in meters: 0.5
Estimated distance between devices in meters: 0.52368877551
Absolute error: 0.0236887755102
Relative error: 0.0473775510204
--------------------
Trial Files: data/audio/trial-0-chirp-0.5m-da.wav and data/audio/trial-0-chirp-0.5m-db.wav
Filter Order: 3
Filter Type: none
Actual distance between devices in meters: 0.5
Estimated distance between devices in meters: 0.558412244898
Absolute error: 0.058412244898
Relative error: 0.116824489796
--------------------
Trial Files: data/audio/trial-0-chirp-0.5

Estimated distance between devices in meters: 0.431092857143
Absolute error: 0.0689071428571
Relative error: 0.137814285714
--------------------
Trial Files: data/audio/trial-2-chirp-0.5m-da.wav and data/audio/trial-2-chirp-0.5m-db.wav
Filter Order: 6
Filter Type: firwin
Actual distance between devices in meters: 0.5
Estimated distance between devices in meters: 0.434951020408
Absolute error: 0.0650489795918
Relative error: 0.130097959184
--------------------
Trial Files: data/audio/trial-2-chirp-0.5m-da.wav and data/audio/trial-2-chirp-0.5m-db.wav
Filter Order: 6
Filter Type: butterworth
Actual distance between devices in meters: 0.5
Estimated distance between devices in meters: 0.434951020408
Absolute error: 0.0650489795918
Relative error: 0.130097959184
--------------------
Trial Files: data/audio/trial-2-chirp-0.5m-da.wav and data/audio/trial-2-chirp-0.5m-db.wav
Filter Order: 6
Filter Type: none
Actual distance between devices in meters: 0.5
Estimated distance between devices in me

Estimated distance between devices in meters: 0.95966122449
Absolute error: 0.0403387755102
Relative error: 0.0403387755102
--------------------
Trial Files: data/audio/trial-1-chirp-1m-da.wav and data/audio/trial-1-chirp-1m-db.wav
Filter Order: 6
Filter Type: none
Actual distance between devices in meters: 1
Estimated distance between devices in meters: 0.924937755102
Absolute error: 0.0750622448979
Relative error: 0.0750622448979
--------------------
Trial Files: data/audio/trial-1-chirp-1m-da.wav and data/audio/trial-1-chirp-1m-db.wav
Filter Order: 9
Filter Type: firwin
Actual distance between devices in meters: 1
Estimated distance between devices in meters: 0.95966122449
Absolute error: 0.0403387755102
Relative error: 0.0403387755102
--------------------
Trial Files: data/audio/trial-1-chirp-1m-da.wav and data/audio/trial-1-chirp-1m-db.wav
Filter Order: 9
Filter Type: butterworth
Actual distance between devices in meters: 1
Estimated distance between devices in meters: 0.959661224

In [ ]:
columns = ["actual_distance",
           "self_distance_a",
           "self_distance_b",
           "device_a_sample_path",
           "device_b_sample_path",
           "tone_frequency0",
           "tone_frequency1",
           "tone_duration",
           "sample_rate",
           "filter_order",
           "filter_type"]

results_table = pd.DataFrame.from_dict(results)[columns]
results_table.to_csv(output_data_directory+"/audio_estimated_distance_results.csv")
results_table